In [8]:
from pathlib import Path
import xarray as xr
import pandas as pd

pathNow = Path().parent.absolute().parent
mswep_dir = pathNow / "Datasets" / "Ina_MSWEP"
gcm_dir   = pathNow / "Datasets" / "Ina_NEX-GDDP-CMIP6-Mahasiswa" / "Adit"

def extract_precip_mswep(start_date, end_date, lat_target, lon_target, output_excel):
    dfs = []

    for year in range(pd.to_datetime(start_date).year, pd.to_datetime(end_date).year + 1):
        file = mswep_dir / f"merged_daily_{year}.nc"
        ds = xr.open_dataset(file)

        # 1. Slice waktu
        ds_time = ds.sel(time=slice(start_date, end_date))

        # 2. Ambil lat lon dengan method="nearest"
        ds_sel = ds_time.sel(lat=lat_target, lon=lon_target, method="nearest")

        df = ds_sel["precipitation"].to_dataframe().reset_index()
        df = df[["time", "precipitation"]]
        df.columns = ["date", "rr"]
        dfs.append(df)

    df_all = pd.concat(dfs)
    df_all.to_excel(output_excel, index=False)

def extract_precip_gcm(start_date, end_date, lat_target, lon_target, output_excel):
    dfs = []

    for year in range(pd.to_datetime(start_date).year, pd.to_datetime(end_date).year + 1):
        file = gcm_dir / f"ina_EC-Earth3_ssp245_{year}_pr.nc"
        ds = xr.open_dataset(file)

        ds_time = ds.sel(time=slice(start_date, end_date))
        ds_sel  = ds_time.sel(lat=lat_target, lon=lon_target, method="nearest")

        pr_mm_day = ds_sel["pr"] * 86400

        df = pr_mm_day.to_dataframe().reset_index()
        df = df[["time", "pr"]]
        df.columns = ["date", "rr"]
        dfs.append(df)

    df_all = pd.concat(dfs)
    df_all.to_excel(output_excel, index=False)

In [9]:
extract_precip_mswep(
    start_date="2015-01-01",
    end_date="2024-12-31",
    lat_target=-6.121074,
    lon_target=106.838433,
    output_excel="MSWEP_2015-2024.xlsx"
)

extract_precip_gcm(
    start_date="2015-01-01",
    end_date="2024-12-31",
    lat_target=-6.121074,
    lon_target=106.838433,
    output_excel="GCM_2015-2024.xlsx"
)


## Akulumasi RR dalam satu hari di STA2172

In [8]:
def sum_daily_rr(filename, target_date):
    df = pd.read_excel(filename)
    
    # Pastikan kolom date dalam datetime
    df['Date'] = pd.to_datetime(df['Date'])

    # Filter hanya tanggal target
    df = df[df['Date'].dt.date == pd.to_datetime(target_date).date()]
    
    # Ambil setiap 10 menit saja (karena nilainya sama)
    df_sampled = df.set_index('Date').resample('10T').first().dropna().reset_index()
    
    # Total RR hari itu
    total_rr = df_sampled['rr'].sum()
    
    return total_rr

In [15]:
# Ganti nama file dan tanggal sesuai kebutuhan
filename = "STA2172_2023-2024.xlsx"
tanggal = "2024-12-23"

total_hujan = sum_daily_rr(filename, tanggal)
print(f"Total RR pada {tanggal} adalah {total_hujan:.2f} mm")

Total RR pada 2024-12-23 adalah 894.80 mm


C:\Users\rifqi\AppData\Local\Temp\ipykernel_27544\840649267.py:11: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  df_sampled = df.set_index('Date').resample('10T').first().dropna().reset_index()
